## merge note lines into full note

#### CONFIGURATIONS

In [ ]:
INPUT_DATASET="<bigquery-project-id>:i2b2_nlp_data"
RESULT_DATASET="<bigquery-project-id>:i2b2_nlp_data"
TARGET_GCS_BUCKET_SHC="<output-bucket>"

GOOGLE_CREDENTIAL="service-account-sample-key.json"

#DATAFLOW CONFIG
DATAFLOW_RUNNER_GCS_BUCKET="<gcs-bucket-for-staging-and-temp-files>"
DATAFLOW_PROJECT="<dataflow-project-id>"
DATAFLOW_DLP_PROJ="<dlp-project-id>"
DATAFLOW_SERVICEACCT="<service-account-email>"
DATAFLOW_MACHINE="n1-standard-8"
DATAFLOW_WORKER="1"

#END OF CONFIGURATION
import subprocess
import os
import json
import sys
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(GOOGLE_CREDENTIAL)


#### EXPORT DEID INPUT DATA TO GCS 
Optional. Deid can also directly read from BigQuery

In [ ]:
%%bash -s "$INPUT_DATASET" "$TARGET_GCS_BUCKET_SHC"
echo "#### NOTE ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$1.training_data' $2/input/i2b2/training_data/text-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$1.testing_data' $2/input/i2b2/testing_data/text-input-*.json"


### run deid on DataFlow

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$DATAFLOW_RUNNER_GCS_BUCKET" "$DATAFLOW_PROJECT" "$DATAFLOW_DLP_PROJ" "$DATAFLOW_SERVICEACCT" "$DATAFLOW_MACHINE" "$DATAFLOW_WORKER" 
cd ..
CMDPRE="mvn -Pdataflow-runner compile exec:java -Dexec.mainClass=com.github.susom.starr.deid.Main -Dexec.args=\"--project=$3 --dlpProject=$4 --serviceAccount=$5 --stagingLocation=$2/staging --gcpTempLocation=$2/temp --tempLocation=$2/temp --region=us-west1 --workerMachineType=$6 --maxNumWorkers=$7 --diskSizeGb=100 --runner=DataflowRunner --deidConfigFile=deid_config_general.yaml --inputType=gcp_gcs  "
echo "#### NOTE ####"
echo "$CMDPRE --textIdFields=\"note_id\" --textInputFields=\"note_text\" --inputResource=$1/input/i2b2/training_data/text-input-*.json --outputResource=$1/i2b2/training_data/DEID_result\""
echo "$CMDPRE --textIdFields=\"note_id\" --textInputFields=\"note_text\" --inputResource=$1/input/i2b2/testing_data/text-input-*.json --outputResource=$2/i2b2/testing_data/DEID_result\""


### load result to BigQuery

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$RESULT_DATASET"
cd ..
CMDPRE="bq --location=US load --autodetect  --source_format=NEWLINE_DELIMITED_JSON "
echo "#### NOTE ####"
echo "$CMDPRE  $2.training_data_deid \"$1/i2b2/training_data/DEID_result/DeidNote-*\" "
echo "$CMDPRE $2.testing_data_deid \"$1/i2b2/testing_data/DEID_result/DeidNote-*\" "
